In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from os import path
from matplotlib import pyplot, colors
import matplotlib.dates as mdates
import matplotlib
import seaborn as sns
import os
import plotly.express as px
# from tobler.area_weighted import area_interpolate
import json
from shapely.geometry import Point, shape

Steps to check for anomalies and clean:

1. Find missing values

2. Check for duplicates

3. Check for leading and trailing whitespaces

4. Check for invalid dates (like the ones where it may be interpreted as 1900-01-01)

5. Save as a new clean CSV to be used for further processing

In [12]:
import pandas as pd
import os

# Define the folder containing the CSV files
folder_path = "raw/spot_prices"

# Define a function to clean a single DataFrame
def clean_data(df):
    # # Step 1: Find and handle missing values
    # missing_values = df.isnull().sum()
    # print("Missing values:\n", missing_values)
    # df = df.dropna()  # Drop rows with missing values (or use df.fillna() to fill them)

    # Step 2: Check for duplicates
    duplicates = df.duplicated().sum()
    print("Duplicate rows:", duplicates)
    df = df.drop_duplicates()

    # Step 3: Check for leading and trailing whitespaces
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()

    # Step 4: Clean and parse the MTU (CET/CEST) column
    # Remove extraneous text like "(CET)" or "(CEST)"
    df["MTU (CET/CEST)"] = df["MTU (CET/CEST)"].str.replace("(CET)", "", regex=False).str.replace("(CEST)", "", regex=False).str.strip()

    # Step 4: Check for invalid dates
    # df["Start Time"] = pd.to_datetime(df["MTU (CET/CEST)"].str.split(" - ").str[0], errors="coerce")
    # df["End Time"] = pd.to_datetime(df["MTU (CET/CEST)"].str.split(" - ").str[1], errors="coerce")
    # invalid_dates = df[df["Start Time"].isnull() | df["End Time"].isnull()]
    # print("Invalid dates:\n", invalid_dates)
    # df = df.dropna(subset=["Start Time", "End Time"])  # Drop rows with invalid dates

    return df

# Iterate through the subfolders (countries) in the folder
output_folder = "processed/spot_prices_cleaned"
os.makedirs(output_folder, exist_ok=True)

for country_folder in os.listdir(folder_path):
    country_path = os.path.join(folder_path, country_folder)
    if os.path.isdir(country_path):
        # Initialize an empty DataFrame to store combined data for the country
        combined_df = pd.DataFrame()
        # Iterate through the CSV files in the country's folder
        for file_name in os.listdir(country_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(country_path, file_name)
                # Read the CSV file
                df = pd.read_csv(file_path)
                # Clean the data
                cleaned_df = clean_data(df)
                # Append the cleaned data to the combined DataFrame
                combined_df = pd.concat([combined_df, cleaned_df], ignore_index=True)

        # Save the combined data for the country to a single CSV file
        output_file = os.path.join(output_folder, f"{country_folder}_processed.csv")
        combined_df.to_csv(output_file, index=False)
        print(f"Processed and saved: {output_file}")

Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Processed and saved: processed/spot_prices_cleaned\AT_processed.csv
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Processed and saved: processed/spot_prices_cleaned\BE_processed.csv
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Processed and saved: processed/spot_prices_cleaned\CZ_processed.csv
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Processed and saved: processed/spot_prices_cleaned\DE_LU_processed.csv
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Processed and saved: processed/spot_prices_cleaned\FR_processed.csv
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Duplicate rows: 0
Processed and saved: processed/spot_prices_cleaned\HR_processed.csv
Duplicate rows: 0
Duplicate rows: 0
Duplicate row

In [22]:
import pandas as pd
import os

# Define the folder containing the processed CSV files
input_folder = "processed/spot_prices_cleaned"
output_folder = "processed/spot_prices_quarter_hours"
os.makedirs(output_folder, exist_ok=True)

# Function to expand rows into quarter-hour intervals
def format_to_quarter_hours(df):
    quarter_hour_rows = []

    for index, row in df.iterrows():
        try:
            # Parse the start and end times
            start_time = pd.to_datetime(row["MTU (CET/CEST)"].split(" - ")[0], dayfirst=True)
            end_time = pd.to_datetime(row["MTU (CET/CEST)"].split(" - ")[1], dayfirst=True)

            # Generate quarter-hour intervals
            current_time = start_time
            while current_time < end_time:
                next_time = current_time + pd.Timedelta(minutes=15)
                quarter_hour_rows.append({
                    "MTU (CET/CEST)": f"{current_time.strftime('%d/%m/%Y %H:%M:%S')} - {next_time.strftime('%d/%m/%Y %H:%M:%S')}",
                    "Area": row["Area"],
                    "Sequence": row["Sequence"],
                    "Day-ahead Price (EUR/MWh)": row["Day-ahead Price (EUR/MWh)"],
                    "Intraday Period (CET/CEST)": row["Intraday Period (CET/CEST)"],
                    "Intraday Price (EUR/MWh)": row["Intraday Price (EUR/MWh)"]
                })
                # print(f"Processing row: {row['MTU (CET/CEST)']}, Current Time: {current_time}")
                current_time = next_time
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue

    # Create a new DataFrame with quarter-hour intervals
    return pd.DataFrame(quarter_hour_rows)

# Iterate through the processed files
for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_folder, file_name)
        # Read the processed CSV file
        df = pd.read_csv(file_path)
        print(f"Formatting file: {file_name}, Total rows: {len(df)}")
        # Format the data into quarter-hour intervals
        formatted_df = format_to_quarter_hours(df)
        # Save the formatted data to a new CSV file
        output_file = os.path.join(output_folder, file_name)
        formatted_df.to_csv(output_file, index=False)
        print(f"Saved formatted file: {output_file}")

Formatting file: AT_processed.csv, Total rows: 350592
Saved formatted file: processed/spot_prices_quarter_hours\AT_processed.csv
Formatting file: BE_processed.csv, Total rows: 43824
Saved formatted file: processed/spot_prices_quarter_hours\BE_processed.csv
Formatting file: CZ_processed.csv, Total rows: 43824
Saved formatted file: processed/spot_prices_quarter_hours\CZ_processed.csv
Formatting file: DE_LU_processed.csv, Total rows: 350592
Saved formatted file: processed/spot_prices_quarter_hours\DE_LU_processed.csv
Formatting file: FR_processed.csv, Total rows: 43824
Saved formatted file: processed/spot_prices_quarter_hours\FR_processed.csv
Formatting file: HR_processed.csv, Total rows: 43824
Saved formatted file: processed/spot_prices_quarter_hours\HR_processed.csv
Formatting file: HU_processed.csv, Total rows: 43824
Saved formatted file: processed/spot_prices_quarter_hours\HU_processed.csv
Formatting file: NL_processed.csv, Total rows: 43824
Saved formatted file: processed/spot_prices

In [23]:
import pandas as pd
import os

# Define the folder containing the raw load files
input_folder = "raw/load"
output_folder = "processed/load_combined"
os.makedirs(output_folder, exist_ok=True)

# Iterate through the subfolders (countries) in the folder
for country_folder in os.listdir(input_folder):
    country_path = os.path.join(input_folder, country_folder)
    if os.path.isdir(country_path):
        # Initialize an empty DataFrame to store combined data for the country
        combined_df = pd.DataFrame()

        # Iterate through the CSV files in the country's folder
        for file_name in os.listdir(country_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(country_path, file_name)
                # Read the CSV file
                df = pd.read_csv(file_path)
                # Append the data to the combined DataFrame
                combined_df = pd.concat([combined_df, df], ignore_index=True)

        # Save the combined data for the country to a single CSV file
        output_file = os.path.join(output_folder, f"{country_folder}_load_combined.csv")
        combined_df.to_csv(output_file, index=False)
        print(f"Processed and saved: {output_file}")

Processed and saved: processed/load_combined\AT_load_combined.csv
Processed and saved: processed/load_combined\BE_load_combined.csv
Processed and saved: processed/load_combined\CZ_load_combined.csv
Processed and saved: processed/load_combined\DE_LU_load_combined.csv
Processed and saved: processed/load_combined\FR_load_combined.csv
Processed and saved: processed/load_combined\HR_load_combined.csv
Processed and saved: processed/load_combined\HU_load_combined.csv
Processed and saved: processed/load_combined\NL_load_combined.csv
Processed and saved: processed/load_combined\PL_load_combined.csv
Processed and saved: processed/load_combined\RO_load_combined.csv
Processed and saved: processed/load_combined\SI_load_combined.csv
Processed and saved: processed/load_combined\SK_load_combined.csv


In [27]:
import pandas as pd
import os

# Define the folder containing the combined load files
input_folder = "processed/load_combined"
output_folder = "processed/load_aggregated"
os.makedirs(output_folder, exist_ok=True)

# Iterate through the combined files for each country
for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_folder, file_name)
        # Read the combined CSV file
        df = pd.read_csv(file_path)

        # Extract the country code from the file name (e.g., "AT" from "AT_load_combined.csv")
        if "DE_LU" in file_name:
            country_code = "DE-LU"
        else:
            country_code = file_name.split("_")[0]

        # Dynamically set column names based on the country code
        day_ahead_column = f"Day-ahead Total Load Forecast [MW] - BZN|{country_code}"
        actual_load_column = f"Actual Total Load [MW] - BZN|{country_code}"

        # Parse the "Time (CET/CEST)" column into start and end times
        df["Start Time"] = pd.to_datetime(df["Time (CET/CEST)"].str.split(" - ").str[0], format="%d.%m.%Y %H:%M")
        df["End Time"] = pd.to_datetime(df["Time (CET/CEST)"].str.split(" - ").str[1], format="%d.%m.%Y %H:%M")

        # Aggregate the data by day (sum of 15-minute intervals per day)
        df["Date"] = df["Start Time"].dt.date  # Extract the date
        daily_sum = df.groupby("Date").agg({
            day_ahead_column: "sum",
            actual_load_column: "sum"
        }).reset_index()

        # Save the aggregated data to a new CSV file
        output_file = os.path.join(output_folder, f"{file_name.replace('_load_combined.csv', '_load_aggregated.csv')}")
        daily_sum.to_csv(output_file, index=False)
        print(f"Aggregated and saved: {output_file}")

Aggregated and saved: processed/load_aggregated\AT_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\BE_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\CZ_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\DE_LU_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\FR_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\HR_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\HU_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\NL_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\PL_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\RO_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\SI_load_aggregated.csv
Aggregated and saved: processed/load_aggregated\SK_load_aggregated.csv


In [28]:
import pandas as pd
import os

# Define the folder containing the raw generation files
input_folder = "raw/generation"
output_folder = "processed/generation_combined"
os.makedirs(output_folder, exist_ok=True)

# Iterate through the subfolders (countries) in the folder
for country_folder in os.listdir(input_folder):
    country_path = os.path.join(input_folder, country_folder)
    if os.path.isdir(country_path):
        # Initialize an empty DataFrame to store combined data for the country
        combined_df = pd.DataFrame()

        # Iterate through the CSV files in the country's folder
        for file_name in os.listdir(country_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(country_path, file_name)
                # Read the CSV file
                df = pd.read_csv(file_path)
                # Append the data to the combined DataFrame
                combined_df = pd.concat([combined_df, df], ignore_index=True)

        # Save the combined data for the country to a single CSV file
        output_file = os.path.join(output_folder, f"{country_folder}_generation_combined.csv")
        combined_df.to_csv(output_file, index=False)
        print(f"Processed and saved: {output_file}")

Processed and saved: processed/generation_combined\AT_generation_combined.csv
Processed and saved: processed/generation_combined\BE_generation_combined.csv
Processed and saved: processed/generation_combined\CZ_generation_combined.csv
Processed and saved: processed/generation_combined\DE_LU_generation_combined.csv
Processed and saved: processed/generation_combined\FR_generation_combined.csv
Processed and saved: processed/generation_combined\HR_generation_combined.csv


C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\2160230102.py:21: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed and saved: processed/generation_combined\HU_generation_combined.csv
Processed and saved: processed/generation_combined\NL_generation_combined.csv
Processed and saved: processed/generation_combined\PL_generation_combined.csv
Processed and saved: processed/generation_combined\RO_generation_combined.csv
Processed and saved: processed/generation_combined\SI_generation_combined.csv
Processed and saved: processed/generation_combined\SK_generation_combined.csv


In [31]:
import pandas as pd
import os

# Define the folder containing the combined generation files
input_folder = "processed/generation_combined"
output_folder = "processed/generation_aggregated"
os.makedirs(output_folder, exist_ok=True)

# Iterate through the combined files for each country
for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_folder, file_name)
        # Read the combined CSV file
        df = pd.read_csv(file_path)

        # Remove "(CET/CEST)" from the MTU column
        df["MTU"] = df["MTU"].str.replace("(CET/CEST)", "", regex=False).str.strip()

        # Parse the cleaned "MTU" column into start and end times
        df["Start Time"] = pd.to_datetime(df["MTU"].str.split(" - ").str[0], format="%d.%m.%Y %H:%M")
        df["End Time"] = pd.to_datetime(df["MTU"].str.split(" - ").str[1], format="%d.%m.%Y %H:%M")

        # Replace "n/e" with 0 for numerical aggregation
        df.replace("n/e", 0, inplace=True)

        # Exclude non-numerical columns from aggregation
        numerical_columns = df.select_dtypes(include=["number"]).columns

        # Aggregate the data by day (sum of 15-minute intervals per day)
        df["Date"] = df["Start Time"].dt.date  # Extract the date
        daily_sum = df.groupby("Date")[numerical_columns].sum().reset_index()

        # Save the aggregated data to a new CSV file
        output_file = os.path.join(output_folder, f"{file_name.replace('_generation_combined.csv', '_generation_aggregated.csv')}")
        daily_sum.to_csv(output_file, index=False)
        print(f"Aggregated and saved: {output_file}")

C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\3378439570.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("n/e", 0, inplace=True)


Aggregated and saved: processed/generation_aggregated\AT_generation_aggregated.csv


C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\3378439570.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("n/e", 0, inplace=True)


Aggregated and saved: processed/generation_aggregated\BE_generation_aggregated.csv


C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\3378439570.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("n/e", 0, inplace=True)


...

C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\3378439570.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("n/e", 0, inplace=True)


Aggregated and saved: processed/generation_aggregated\RO_generation_aggregated.csv


C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\3378439570.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("n/e", 0, inplace=True)


Aggregated and saved: processed/generation_aggregated\SI_generation_aggregated.csv
Aggregated and saved: processed/generation_aggregated\SK_generation_aggregated.csv


C:\Users\Amic\AppData\Local\Temp\ipykernel_13672\3378439570.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("n/e", 0, inplace=True)
